# Real-Time Experimental Results Visualizer

## Setup

In [1]:
from code.excelmem.excelmemvis import main as excel_main
from code.libremem.librememvis import main as libre_main

import plotly.graph_objects as go
import multiprocessing as mp

In [2]:
fv_fig = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Formula-Value Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Formula Size (MB)")
    )
)
fv_fig.add_scatter(name='Formula-Value (MB)')
fv_fig

FigureWidget({
    'data': [{'name': 'Formula-Value (MB)', 'type': 'scatter', 'uid': 'c2e40d87-8885-4450-8aca-…

In [3]:
vo_fig = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Value-Only Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Value Size (MB)")
    )
)
vo_fig.add_scatter(name='Value-Only (MB)')
vo_fig

FigureWidget({
    'data': [{'name': 'Value-Only (MB)', 'type': 'scatter', 'uid': '08b662ee-9a97-425f-996a-ccf…

In [4]:
barplt = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Size (MB)")
    )
)
barplt.add_bar(name='Value-Only (MB)')
barplt.add_bar(name='Formula-Value (MB)')
barplt

FigureWidget({
    'data': [{'name': 'Value-Only (MB)', 'type': 'bar', 'uid': 'd1132807-3624-4766-817f-80e24f6…

## Place your own settings here

In [5]:
# Either "libre" or "excel"
APPLICATION = "libre"

SOFFICE_DIR = "C:/Program Files/LibreOffice/program/soffice"
INPUTS_PATH = os.path.join("input-data", "rscs-test")
OUTPUT_PATH = os.path.join(OUTPUT_NAME, APPLICATION.lower(), OUTDIR_NAME)
FV_INPUTDIR = "formula-value"
VO_INPUTDIR = "value-only"
INTEGER_ARG = 1

## Run the following cells for real-time results

In [6]:
parent_conn, child_conn = mp.Pipe()

if APPLICATION.lower() == "excel":
  process = mp.Process(target=excel_main
    , args=(child_conn, INPUTS_PATH, OUTPUT_PATH,)
    , kwargs={
      "fv_inputdir"   : FV_INPUTDIR
      , "vo_inputdir" : VO_INPUTDIR
      , "totl_trials" : INTEGER_ARG
    }
  )

elif APPLICATION.lower() == "libre":
  process = mp.Process(target=libre_main
    , args=(child_conn, INPUTS_PATH, OUTPUT_PATH,)
    , kwargs={
      "soffice_path"  : SOFFICE_DIR
      , "fv_inputdir" : FV_INPUTDIR
      , "vo_inputdir" : VO_INPUTDIR
      , "pollseconds" : INTEGER_ARG
    }
  )

else:
  process = None
  print(f"{APPLICATION} is an invalid option.")

In [7]:
if process is not None:
    process.start()
    while True:
        item = parent_conn.recv()
        if type(item) == dict:
            vo_rowsizes = []; vo_memsizes = []; fv_rowsizes = []; fv_memsizes = []
            for r, d in sorted(item.items(), key=lambda pair: pair[0]):
                if "Value uss (MB)"   in d: vo_rowsizes.append(str(r)); vo_memsizes.append(d["Value uss (MB)"  ])
                if "Formula uss (MB)" in d: fv_rowsizes.append(str(r)); fv_memsizes.append(d["Formula uss (MB)"])
            barplt.data[0].x = vo_fig.data[0].x = vo_rowsizes
            barplt.data[1].x = fv_fig.data[0].x = fv_rowsizes
            barplt.data[0].y = vo_fig.data[0].y = vo_memsizes
            barplt.data[1].y = fv_fig.data[0].y = fv_memsizes
        if type(item) == str:   print(item)
        if item is None:        break
    process.join()
    process.terminate()

: 321110016, 'pagefile': 127553536, 'private': 127553536, 'wset': 321105920, 'rss': 321105920, 'uss': 104886272, 'vms': 127553536}
vo-50000.ods: {'peak_nonpaged_pool': 62768, 'peak_paged_pool': 1548936, 'num_page_faults': 183658, 'peak_pagefile': 482476032, 'nonpaged_pool': 62496, 'paged_pool': 1548928, 'peak_wset': 561901568, 'pagefile': 482291712, 'private': 482291712, 'wset': 561897472, 'rss': 561897472, 'uss': 339148800, 'vms': 482291712}
vo-50000.ods: {'peak_nonpaged_pool': 62768, 'peak_paged_pool': 1548992, 'num_page_faults': 183681, 'peak_pagefile': 482476032, 'nonpaged_pool': 62360, 'paged_pool': 1547184, 'peak_wset': 561995776, 'pagefile': 482258944, 'private': 482258944, 'wset': 561975296, 'rss': 561975296, 'uss': 339226624, 'vms': 482258944}
vo-50000.ods: {'peak_nonpaged_pool': 62768, 'peak_paged_pool': 1548992, 'num_page_faults': 183681, 'peak_pagefile': 482476032, 'nonpaged_pool': 62360, 'paged_pool': 1547184, 'peak_wset': 561995776, 'pagefile': 482258944, 'private': 48225

In [8]:
# Timing notes

# EXCEL:
# large-RCBS-1col (Python)
#   Up to 1 million: Total time (HH:MM:SS): 06:56:18
# RCBS-1col (Python)
#   Up to 1 million: Total time (HH:MM:SS): 07:05:28
# RCBS-1col (VBA)
#   Up to 1 million: Total time (HH:MM:SS): 07:47:56
# RLS-1col
#   Up to 1 million: Total time (HH:MM:SS): 00:06:18
# RSCS-1col (VBA)
#   Up to 1 million: Total time (HH:MM:SS): 00:08:18
# RLS-5col
#   Up to 1 million: Total time (HH:MM:SS): 00:21:21